# PySpark

# Multiprocessing

# Pathos

# Dask

# HDFS y Parquet  (sistemas de archivos)

# tqdm

# Vaex

https://towardsdatascience.com/how-to-analyse-100s-of-gbs-of-data-on-your-laptop-with-python-f83363dda94

**Data:** https://vaex.io/docs/datasets.html

**si no plotea...:** https://stackoverflow.com/questions/57323126/how-to-do-interactive-2d-scatter-plot-zoom-point-selection-in-vaex